In [60]:
import yfinance as yf
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


ticker = 'AAPL'  # ticker symbol
data = yf.download(ticker, period='3Y', progress=False)  # Adjust the date range as needed

data['DayOfWeek'] = data.index.dayofweek
data['DayOfYear'] = data.index.dayofyear
data['month'] = data.index.month

returns = np.log(data['Adj Close']/data['Adj Close'].shift()).bfill()  # Calculate log returns
returns = torch.tensor(returns.values).float().unsqueeze(1)  # Convert to tensor

normalized_features = data[['DayOfWeek', 'DayOfYear', 'month']].ffill()
normalized_features = (normalized_features - normalized_features.mean()) / normalized_features.std()

class VolatilityDataset(Dataset):
    def __init__(self, returns, features):
        self.returns = returns
        self.features = features

    def __len__(self):
        return len(self.returns) - 1
    
    def __getitem__(self, index):
        return self.returns[index], self.features[index]

# Step 2: Design the neural network architecture
class GARCHNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GARCHNet, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)

    def forward(self, x, features):
        _, h = self.rnn(x)
        out = self.fc1(torch.cat((h[-1], features), dim=1))  # Use the final RNN output for volatility prediction
        return out

# Step 3: Define GARCH equations and loss function
def garch_loss(volatility, returns):
    sigma_sq = volatility**2
    log_likelihood = -0.5 * (torch.log(sigma_sq) + (returns**2 / sigma_sq))
    return -torch.mean(log_likelihood)

# Step 4: Prepare training and validation data
train_ratio = 0.8  # Training-validation split ratio
train_size = int(train_ratio * len(returns))
train_returns = returns[:train_size]
train_features = normalized_features[:train_size]

val_returns = returns[train_size:]
val_features = normalized_features[train_size:]

train_dataset = VolatilityDataset(train_returns[:-1], train_features[:-1])
val_dataset = VolatilityDataset(val_returns[:-1], val_features[:-1])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Step 5: Train the combined model
input_size = 1  # Number of input features (1 for returns)
hidden_size = 32  # Number of hidden units in the RNN
output_size = 1  # Number of output units (1 for volatility)

model = GARCHNet(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    model.train()

    for returns_batch, features_batch in train_loader:
        optimizer.zero_grad()

        volatility = model(returns_batch, features_batch)  # Predict volatility using the model
        loss = criterion(volatility.squeeze(), returns_batch.squeeze())  # Compare against the next day's returns
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Step 6: Evaluate the model on the validation set
val_loss = 0.0
with torch.no_grad():
    for returns_batch, features_batch in val_loader:
        volatility = model(returns_batch, features_batch)
        loss = criterion(volatility.squeeze(), returns_batch.squeeze())
        val_loss += loss.item() * returns_batch.size(0)  # Accumulate the loss over batches

val_loss /= len(val_dataset)  # Calculate the average loss
print(f'Validation Loss: {val_loss}')

# Step 7: Use the trained model for volatility forecasting (example with one-step ahead)
model.eval()
forecasted_volatility = []

with torch.no_grad():
    for i in range(len(val_returns) - 1):
        returns_input = val_returns[i].unsqueeze(0)
        features_input = val_features[i].unsqueeze(0)
        volatility = model(returns_input, features_input)
        forecasted_volatility.append(volatility.item())

forecasted_volatility = torch.tensor(forecasted_volatility)

KeyError: 148